In [ ]:
import os
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas_ta as ta
import datetime
from logzero import logger, loglevel
import logzero
import st2

In [ ]:
data = pd.read_csv('../nifty_200/5M/SBIN.csv')

In [ ]:
data.Datetime = pd.to_datetime(data.Datetime)
data.set_index('Datetime', inplace=True)

In [ ]:
data_hourly = st2.custom_resample(data,freq='1h')

In [ ]:
data_hourly['RSI'] = ta.rsi(data_hourly['Close'], timeperiod=14)


In [ ]:
data['Date'] = data.index.date

In [ ]:
data['datetime'] = data.index

In [ ]:
data_hourly

In [ ]:
merged_Data = pd.merge(data, data_hourly[['RSI']], left_index=True, right_index=True, how='left').ffill()

In [ ]:
daily_data = st2.custom_resample(data, freq = 'D')
daily_data['Date'] = daily_data.index.date
daily_data = st2.apply_pivots(daily_data)


In [ ]:
daily_data

In [ ]:
merged_Data = pd.merge(merged_Data, daily_data[['Date', 'PP', 'TC', 'BC', 'R1', 'R2', 'R3', 'S1', 'S2', 'S3', '%CPR']], left_on='Date', right_on='Date', how='left')

In [ ]:
merged_Data.set_index('datetime', inplace=True)


In [ ]:
merged_Data['wma'] = ta.vwma(merged_Data['Close'], merged_Data['Volume'], length=5)

In [ ]:
def define_trading_conditions(data):
    """Define the buy and sell conditions."""
    start_time = datetime.time(10, 14)
    end_time = datetime.time(14, 16)
    buy_condition = (
        (data['%CPR'] < 0.01) &
        (data.RSI > 70) &
        (data.Close > data.R1) &
        (data.Close.shift(1) < data.R1) &
        (data.index.time >= start_time) &
        (data.index.time <= end_time)
    )
                    
    sell_condition = (data.Close > data.R3) | (data.Close < data.TC) 
    
    data['Signal'] = np.select([buy_condition, sell_condition], ['Buy', 'Sell'])
    data['Shifted_close'] = data['Close'].shift()
    
    
    return data

In [ ]:
signal_df = define_trading_conditions(merged_Data)

In [ ]:
signal_df

In [ ]:
def backtest(df, initial_capital=100000, risk_per_trade=0.01, stop_loss=0.01, take_profit=0.03):
    
    """Backtest the trading strategy with given conditions."""
    position = False
    trades = {'Buy Date': [], 'Buy Price': [], 'Sell Date': [], 'Sell Price': [], 'Quantity': []}
    capital = initial_capital
    close_time = datetime.time(15, 10)
    for index, row in df.iterrows():
        
        if row['Signal'] == 'Buy' and not position:
            position = True
            risk_amount = capital * risk_per_trade
            qty = math.floor(risk_amount / (stop_loss * row['Close']))
            capital -= qty * row['Close']
            trades['Buy Date'].append(index)
            trades['Buy Price'].append(row['Close'])
            trades['Quantity'].append(qty)
            logger.info(f"Buy Signal: {index}, Price: {row['Close']}, Quantity: {qty}")

        if position:
            if row['Signal'] == 'Sell' or row['Shifted_close'] < (1 - stop_loss) * trades['Buy Price'][-1]  or row['Shifted_close'] > (1 + take_profit) * trades['Buy Price'][-1] or index.time() > close_time:
                trades['Sell Date'].append(index)
                trades['Sell Price'].append(row['Close'])
                capital += qty * row['Close']
                logger.info(f"Sell Signal: {index}, Price: {row['Close']}, PnL: {qty * (row['Close'] - trades['Buy Price'][-1])}")
                position = False

    # Ensure lists are of equal length by filling with NaN
    max_len = max(len(trades['Buy Date']), len(trades['Sell Date']))
    for key in trades.keys():
        trades[key].extend([np.nan] * (max_len - len(trades[key])))

    return pd.DataFrame(trades)

# Metrics Calculation and Visualization

def calculate_trade_metrics(trade_history_df):
    """Calculate metrics like Return, PnL, Cumulative Profit, etc."""
    trade_history_df['Return'] = (trade_history_df['Sell Price'] - trade_history_df['Buy Price']) / trade_history_df['Buy Price']
    trade_history_df['Days'] = (trade_history_df['Sell Date'] - trade_history_df['Buy Date']).dt.days
    trade_history_df['PnL'] = trade_history_df['Sell Price'] - trade_history_df['Buy Price']
    trade_history_df['Realized Profit'] = trade_history_df['PnL'] * trade_history_df['Quantity']
    trade_history_df['Cum Profit'] = trade_history_df['Realized Profit'].cumsum()
    trade_history_df['Cumulative Return'] = (1 + trade_history_df['Return']).cumprod() - 1
    trade_history_df['Drawdown'] = trade_history_df['Cum Profit'].cummax() - trade_history_df['Cum Profit']
    
    # Calculate KPIs
    total_trades = len(trade_history_df.dropna())
    winning_trades = len(trade_history_df[trade_history_df['PnL'] > 0].dropna())
    losing_trades = len(trade_history_df[trade_history_df['PnL'] < 0].dropna())
    win_rate = winning_trades / total_trades if total_trades > 0 else 0
    avg_pnl = trade_history_df['PnL'].mean()
    total_profit = trade_history_df['Realized Profit'].sum()
    
    kpi = {
        'Total Trades': total_trades,
        'Winning Trades': winning_trades,
        'Losing Trades': losing_trades,
        'Win Rate': win_rate,
        'Average PnL': avg_pnl,
        'Total Profit': total_profit,
        'Max Drawdown': trade_history_df['Drawdown'].max()
    }
    
    return trade_history_df, kpi


In [ ]:
backtest_results = backtest(signal_df)

In [ ]:
backtest_results

In [ ]:
trade_history_df, kpi = calculate_trade_metrics(backtest_results)

In [ ]:
trade_history_df

In [ ]:
def plot_metrics(trade_history_df, stock_symbol):
    """Plot cumulative profit and drawdown."""
    plt.figure(figsize=(14, 7))

    # Plot Cumulative Profit
    plt.subplot(2, 1, 1)
    plt.plot(trade_history_df['Buy Date'], trade_history_df['Cum Profit'], label='Cumulative Profit')
    plt.title(f'Cumulative Profit for {stock_symbol}')
    plt.xlabel('Trade')
    plt.ylabel('Cumulative Profit')
    plt.legend()

    # Plot Drawdown
    plt.subplot(2, 1, 2)
    plt.plot(trade_history_df.index, trade_history_df['Drawdown'], label='Drawdown', color='red')
    plt.title(f'Drawdown for {stock_symbol}')
    plt.xlabel('Trade')
    plt.ylabel('Drawdown')
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
plot_metrics(trade_history_df, "ABB")

In [ ]:
kpi

In [ ]:
trade_history_df.loc[trade_history_df['Realized Profit'].idxmax()]

In [67]:
pd.read_csv('../FnO.csv')

,Index,SYMBOL,OPEN \n,HIGH \n,LOW \n,PREV. CLOSE \n,LTP \n,CHNG \n,%CHNG \n,VOLUME \n(shares),VALUE \n (₹ Crores),52W H \n,52W L \n,30 D %CHNG \n,365 D % CHNG \n 17-Aug-2023
0,1,PEL,885,951.75,880.85,882.4,944,61.6,6.98,"41,37,386",381.6,"1,139.95",736.6,-3.12,-10.73
1,2,MPHASIS,"2,726.05","2,918.50","2,726.05","2,718.85","2,908.00",189.15,6.96,"17,86,753",513.61,"3,080.95","2,069.10",3.98,24.4
2,3,BSOFT,576.05,606.75,572.2,566,602,36,6.36,"95,97,224",571.98,861.85,446.25,-17.39,27.77
3,4,ABCAPITAL,205,216.98,205,203.47,215.85,12.38,6.08,"1,05,87,128",225.68,246.9,155,-4.14,19.58
4,5,MCX,"4,379.55","4,669.00","4,379.05","4,379.55","4,625.00",245.45,5.60,"11,83,924",543.31,"4,669.00","1,538.00",17.36,194.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,177,DIVISLAB,"4,665.00","4,709.80","4,623.30","4,662.00","4,633.15",-28.85,-0.62,"5,22,991",243.25,"5,024.85","3,295.30",1.38,26.79
177,178,IPCALAB,"1,369.80","1,369.80","1,314.10","1,349.05","1,336.15",-12.9,-0.96,"4,52,124",60.23,"1,413.70",856.8,9.83,50.59
178,179,VOLTAS,"1,570.00","1,592.00","1,526.90","1,550.30","1,531.75",-18.55,-1.20,"26,24,760",406.85,"1,598.90",800,0.88,85.91
179,180,GRANULES,673.05,677.8,659.1,669.3,660,-9.3,-1.39,"9,61,779",64.24,697.55,288,27.17,125.25
